In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment.util import *
from nltk.sentiment import SentimentAnalyzer
from datetime import datetime
from datetime import timedelta
import csv
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
from statistics import mean
import csv
import os


#################################################
#                                               #
#     Sentiment helper function definitions     #
#                                               #
#################################################

#companies with noisy names are exluded
exclusions = ["WOOD", "BEST", "BILL", "TV", "DD", "CARS", "TRUE", "PSA", "LOW"]

date_format = "%Y-%m-%d"

#takes a reddit posts and returns the datetime
def _date(x):
  date_time = (x.str.split(", "))[0]
  date = (date_time[0].split(" "))[0]
  a = datetime.strptime(date, date_format)
  return a

#if x is an earlier date than y then true
#if x is equal or later then false
def compare(x, y):
  if(_date(x) < _date(y)):
    return True
  else:
    return False

#sorts reddit posts by date
def sort_posts(posts):
  sorted = posts
  for i in range(1, len(posts)):
    j = i
    while(j>0):
      if(sorted.iloc[j].empty):
        del sorted.iloc[j]
        j = j - 1
      elif(compare(sorted.iloc[j], sorted.iloc[j-1])):
        tmp = sorted.iloc[j-1]
        sorted.iloc[j-1] = sorted.iloc[j]
        sorted.iloc[j] = tmp
        j = j - 1
      else:
        j = j - 1
  return sorted

def sort_posts2(posts):
  return posts[::-1]

#returns the title of a reddit posts
def get_title(post):
    split1 = (post.str.split(",")).iloc[1]
    return split1

#splits a text into sentences and returns the mean compund score
#this is where the magic happens
def mean_score(input):
    sentences = input.split(". ")
    scores = [
        sia.polarity_scores(sentence)["compound"]
        for sentence in sentences
    ]
    scores_neg = [
        sia.polarity_scores(sentence)["neg"]
        for sentence in sentences
    ]
    scores_pos = [
        sia.polarity_scores(sentence)["pos"]
        for sentence in sentences
    ]
    return mean(scores), max(scores), min(scores), mean(scores_neg), mean(scores_pos)

#takes a set of companies to analies so we dont have to reinitialize vader each time
def sentiment_main(input_file):
  df = pd.read_csv(input_file[0])
  df.columns = ['Date', 'Ticker', 'Title', 'Selftext']
  posts_sorted = sort_posts2(df) 
  ticker = input_file[1]
  if(len(df) <= 100 or ticker in exclusions):
    return 0
  subreddit = input_file[2]
  sent_result = sentiment_loop(ticker, posts_sorted, subreddit)
  if(sent_result == 0):
    print("weird yfinance error")
  return 1

def sentiment_loop(ticker, posts, subreddit):
  #get stock data
  googl = yf.Ticker(ticker)
  #get date of first market day
  first = _date(posts.iloc[0])
  first_str = first.strftime("%Y-%m-%d")
  try:
    history = googl.history(start=first_str, interval="1d")
  except:
    return 0
  #convert stock into a dataframe
  gld_df = pd.DataFrame(history)
  open_df = list(gld_df["Open"])
  dates = gld_df.index
  #open market days
  dates_new = []
  for date in dates:
    dates_new.append(date.date())
  #mean score on a given day
  scores = np.zeros(len(dates))
  highest_compound = np.zeros(len(dates))
  lowest_compound = np.zeros(len(dates))
  pos_mean = np.zeros(len(dates))
  neg_mean = np.zeros(len(dates))
  #number of posts on a given date
  num_posts = np.zeros(len(dates))
  for index, row in posts.iterrows(
    try:
      reddit_date = datetime.strptime(row[0], "%Y-%m-%d %H:%M:%S")
    except:
      print("invalid date format seen")
      continue
    #open market day
    if(reddit_date.date() in dates_new):
      for index, date in enumerate(dates_new):
        if(reddit_date.date() == date):
          #date matched
          if(isinstance(row["Selftext"], str) ):
            #mean of compound score
            scores[index], highest_compound[index], lowest_compound[index], neg_mean[index], pos_mean[index] = (scores[index] * num_posts[index] + mean_score(row["Selftext"]))/(num_posts[index] + 1)
            #number of posts
            num_posts[index] = num_posts[index] + 1
          break
    #closed market day
    else:
      for index, date in enumerate(dates_new):
        if(reddit_date.date() < date and reddit_date.date() > dates_new[index-1]):
          #date matched
          if(isinstance(row["Selftext"], str) ):
            #mean of compound score
            scores[index], highest_compound[index], lowest_compound[index], neg_mean[index], pos_mean[index] = (scores[index] * num_posts[index] + mean_score(row["Selftext"]))/(num_posts[index] + 1)
            #number of posts
            num_posts[index-1] = num_posts[index-1] + 1
          break
  with open("/content/drive/MyDrive/SENG474_Project/data/historical/s_"+subreddit+"/"+subreddit+"_"+ticker+"_sentiment.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["date", "CompoundMean", "NumPosts", "highestCompund", "lowestCompound", "negMean", "posMean"])
    for i in range(0, len(dates_new)):   
      writer.writerow([dates_new[i], scores[i], num_posts[i], highest_compound[i], lowest_compound[i], pos_mean[i], neg_mean[i]])
  return 1



#initiaalize vader, this is global because multiple functions access it and I dont want to reinitialize it
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
sia = SIA()
#vader custom words
new_words = {
  'undervalued' : 1,
  'bull' : 1,
  'bullish' : 1,
  'overvalued' :  -1,
  'bear' :  -1,
  'bearish' : -1,
}


sia.lexicon.update(new_words)
#input_files = [["/content/drive/MyDrive/SENG474_Project/data/investing/investing_AAPL.csv", "TSLA", "investing"],
#               ["/content/drive/MyDrive/SENG474_Project/data/investing/investing_AMZN.csv", "AMZN", "investing"],
#               ["/content/drive/MyDrive/SENG474_Project/data/investing/investing_FB.csv", "FB", "investing"],
#               ["/content/drive/MyDrive/SENG474_Project/data/investing/investing_GOOG.csv", "GOOG", "investing"],
#               ["/content/drive/MyDrive/SENG474_Project/data/investing/investing_MSFT.csv", "MSFT", "investing"],
##               ["/content/drive/MyDrive/SENG474_Project/data/investing/investing_TSLA.csv", "TSLA", "investing"],
#              ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_AAPL.csv", "AAPL", "stocks"],
#               ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_AMZN.csv", "AMZN", "stocks"],
#               ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_FB.csv", "FB", "stocks"],
#               ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_GOOG.csv", "GOOG", "stocks"],
#               ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_MSFT.csv", "MSFT", "stocks"],
#               ["/content/drive/MyDrive/SENG474_Project/data/stocks/stocks_TSLA.csv", "TSLA", "stocks"],]
#sentiment_main(input_files)

directory_list = ["/content/drive/MyDrive/SENG474_Project/data/historical/r_stocks", "/content/drive/MyDrive/SENG474_Project/data/historical/r_investing", "/content/drive/MyDrive/SENG474_Project/data/historical/r_wsb"]
for directory in directory_list:
  print(directory)
  for filename in os.listdir(directory):
    filename_split = filename.split("_")
    input = [directory+"/"+filename, filename_split[1], filename_split[0]]
    result = sentiment_main(input)
    if(result == 1):
      print("done "+filename)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/content/drive/MyDrive/SENG474_Project/data/historical/r_stocks
done stocks_GILD_r.csv
done stocks_MRNA_r.csv
done stocks_DEEP_r.csv
done stocks_SHOP_r.csv
done stocks_AAL_r.csv
done stocks_TWTR_r.csv
done stocks_UBER_r.csv
done stocks_PM_r.csv
done stocks_FLOW_r.csv
done stocks_INO_r.csv
done stocks_SGLB_r.csv
done stocks_NAT_r.csv
done stocks_UA_r.csv
done stocks_REIT_r.csv
done stocks_DIS_r.csv
done stocks_PENN_r.csv
done stocks_SBUX_r.csv
done stocks_GME_r.csv
done stocks_ATVI_r.csv
done stocks_VOO_r.csv
done stocks_MAR_r.csv
done stocks_PLUG_r.csv
done stocks_CVS_r.csv
done stocks_DAL_r.csv
done stocks_PFE_r.csv
done stocks_WMT_r.csv
done stocks_INTC_r.csv
done stocks_MAN_r.csv
done stocks_AGE_r.csv
done stocks_NOK_r.csv
done stocks_PEAK_r.csv
done stocks_TTWO_r.csv
done stocks_EA_r.csv
done stocks_MU_r.csv
done stocks_IBIO_r.csv
done stocks_AMC_r.csv
done

In [ ]:
directory_list = ["/content/drive/MyDrive/SENG474_Project/data/historical/r_investing"]
for directory in directory_list:
  print(directory)
  for filename in os.listdir(directory):
    print(filename)